### Here's a Markdown [cheatsheet](https://medium.com/ibm-data-science-experience/markdown-for-jupyter-notebooks-cheatsheet-386c05aeebed)


In [1]:
import numpy as np
import matplotlib.pyplot as plt


## Read part of a wav file with Librosa.

### librosa `load()` parameters:
-    `sr`: use sr=None to preserve the "native" rate of the file
-    `offset` and `duration` are measured in **seconds**
    -    one channel = mono, two = stereo
-    `mono`: librosa converts to mono by default
    -    as you can see in the next cell, conversion from stereo to mono averages the two channels

### librosa `load()` return value (a pair):
-    a waveform `y`: a numpy.ndarry of dimension (n,) or **(2,n)**
    -  note: each channel is one **row**
    -  it appears that by default, each sample is a float in the range (-1, 1)
-    the sampling rate `sr`


In [2]:
import librosa

#    read both channels of a stereo wav file
#
offset=0.0 # seconds
duration=20.0 # seconds
ylib_stereo, srlib_stereo = librosa.load('/Akamai/voice/data/Basiani/01 Mravalzhamier - Table Song, Kakheti.wav', 
                     mono=False, sr=None, offset=offset, duration=duration)

start_sample=1000
num_samples=10
print('Stereo')
print(ylib_stereo.shape)
print(ylib_stereo[:,start_sample:start_sample+num_samples])

# average the two channels
#
print('\nAverage of stereo channels')
z = np.add(np.ndarray.flatten(ylib_stereo[0,start_sample:start_sample+num_samples]), 
           np.ndarray.flatten(ylib_stereo[1,start_sample:start_sample+num_samples]))/2
print(z)


#    read mono from a stereo wav file
#
ylib, srlib = librosa.load('/Akamai/voice/data/Basiani/01 Mravalzhamier - Table Song, Kakheti.wav', 
                     mono=True, sr=None, offset=0.0, duration=20)

print('\nMono')
print(ylib.shape)
print(ylib[start_sample:start_sample+num_samples])

Stereo
(2, 882000)
[[-6.1035156e-05 -1.2207031e-04 -6.1035156e-05 -9.1552734e-05
  -9.1552734e-05 -9.1552734e-05 -3.0517578e-05 -6.1035156e-05
  -9.1552734e-05 -6.1035156e-05]
 [-6.1035156e-05 -9.1552734e-05 -9.1552734e-05 -9.1552734e-05
  -9.1552734e-05 -6.1035156e-05 -3.0517578e-05 -6.1035156e-05
  -9.1552734e-05 -3.0517578e-05]]

Average of stereo channels
[-6.1035156e-05 -1.0681152e-04 -7.6293945e-05 -9.1552734e-05
 -9.1552734e-05 -7.6293945e-05 -3.0517578e-05 -6.1035156e-05
 -9.1552734e-05 -4.5776367e-05]

Mono
(882000,)
[-6.1035156e-05 -1.0681152e-04 -7.6293945e-05 -9.1552734e-05
 -9.1552734e-05 -7.6293945e-05 -3.0517578e-05 -6.1035156e-05
 -9.1552734e-05 -4.5776367e-05]


## Read part of a wav file with Soundfile.
### soundfile `read()` parameters:
-    `samplerate`: use samplerate=None (the default) to preserve the "native" rate of the file
-    `start` and `frames` are measured in **frames** (not seconds): one frame is one sample of each *channel* 
    -    one channel = mono, two = stereo

### soundfile `read()` return value (a pair):
-    a waveform `y`: a numpy.ndarry of dimension (n,) or **(n,2)**
    -  note: each channel is one **column**
    -  by default, each sample is a float in the range (-1, 1)
-    the sampling rate `sr`


In [3]:
import soundfile

start=int(offset)*srlib
frames=int(duration)*srlib

ysf, srsf = soundfile.read('/Akamai/voice/data/Basiani/01 Mravalzhamier - Table Song, Kakheti.wav', 
                       samplerate=None, start=start, frames=frames)

print(ysf.shape)
print(ysf[start_sample:start_sample+num_samples])

(882000, 2)
[[-6.10351562e-05 -6.10351562e-05]
 [-1.22070312e-04 -9.15527344e-05]
 [-6.10351562e-05 -9.15527344e-05]
 [-9.15527344e-05 -9.15527344e-05]
 [-9.15527344e-05 -9.15527344e-05]
 [-9.15527344e-05 -6.10351562e-05]
 [-3.05175781e-05 -3.05175781e-05]
 [-6.10351562e-05 -6.10351562e-05]
 [-9.15527344e-05 -9.15527344e-05]
 [-6.10351562e-05 -3.05175781e-05]]


## Write both input segments to files using soundfile `write()`

### Note: Librosa `write_wav()` is *deprecated*, meaning it will be removed from the library soon.

#### First, write the segment read with Soundfile

In [13]:
file = '/Akamai/voice/data/Basiani/01 Mravalzhamier/' + str(offset) + '-' + str(offset+duration) + '.wav'
print(ysf.shape)
soundfile.write(file, ysf, srsf)


(882000, 2)


#### Second, write the segment read with Librosa, after transposing it.

In [12]:
file = '/Akamai/voice/data/Basiani/01 Mravalzhamier/' + str(offset) + '-' + str(offset+duration) + '.lib.wav'
print(ylib_stereo.T.shape)
soundfile.write(file, ylib_stereo.T, srlib)


(882000,)


#### Read an mp3 with Librosa, and write it as a wav.

In [7]:
from os.path import join, splitext

file = join('/Akamai/voice/data/', 'Scherbaum Mshavanadze/')
file = join(file, 'GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919')
file = join(file, 'GVM009_BatonebisNanina_Tbilisi_Mzetamze_20160919_AHDS1M.mp3')
y_mp3, sr_mp3 = librosa.load(file, sr=None)
print(ylib_stereo.T.shape)
soundfile.write(splitext(file)[0] + '.wav', y_mp3.T, sr_mp3)



/opt/anaconda3/lib/python3.7/site-packages/librosa/core/audio.py:161: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn('PySoundFile failed. Trying audioread instead.')


(882000, 2)
